# SULT

## Standard ultimate life table

This tabulates single net premiums and basic functions (whole life and endowment insurances,  whole life and temporary annuities and pure endowments) for several time periods at integer ages between 20 and 100 years. According to the SOA's "Excel Workbook for FAM-L Tables", this table was developed from the following assumptions:

- constant interest rate $i=0.05$
- radix of 100000 initial lives aged 20
- incorporates Makeham's Law as its survival model with $A = 0.00022,~ B = 0.0000027, ~ c = 1.124$


## Pure endowment

Pure endowment functions can be calculated from numbers of lives survived and compounded interest rates.

$_tE_x = v^t ~ \dfrac{l_{x+t}}{l_x}$

${^2_t}E_x = v^{2t} ~ \dfrac{l_{x+t}}{l_x} = v^t ~ _tE_x$

## Term life insurance

Term life insurance functions can be calculated from whole life insurance and pure endowment table columns.

$A^ {1}_{x:\overline{t|}} = A_x - ~_tE_x ~ A_{x+t} = A_{x:\overline{t|}} - ~_tE_x$

${^2}A^ {1}_{x:\overline{t|}} = {^2}A_x - ~{^2_t}E_x ~ {^2}A_{x+t} = {^2}A_x - ~v^{t} ~ _tE_x ~ {^2}A_{x+t}$


## Methods


The `SULT` class implements an instance of a LifeTable, called the standard ultimate life table, which is based on Makeham's Law with parameters specified in SOA's "Excel Workbook for FAM-L Tables"

In [1]:
import math
from actuarialmath import SULT
import describe
describe.methods(SULT)


class SULT - Generates and uses a standard ultimate life table

    Args:
      i : interest rate
      radix : initial number of lives
      minage : minimum age
      maxage : maximum age
      S : survival function, default is Makeham with SOA FAM-L parameters

    Examples:
      >>> sult = SULT()
      >>> a = sult.temporary_annuity(70, t=10)
      >>> A = sult.deferred_annuity(70, u=10)
      >>> P = sult.gross_premium(a=a, A=A, benefit=100000, initial_premium=0.75,
      >>>                        renewal_premium=0.05)

    Methods:
    --------

    frame(minage, maxage):
      Derive FAM-L exam table columns of SULT as a DataFrame

    __getitem__(col):
      Returns a column of the sult table




## Examples

__SOA Question 6.52__

For a fully discrete 10-payment whole life insurance of H on (45), you are given:
- Expenses payable at the beginning of each year are as follows:

| Expense Type | First Year | Years 2-10 | Years 11+ |
|---|---|---|---|
| Per policy | 100 | 20 | 10 |
| % of Premium |105% | 5% | 0% |

- Mortality follows the Standard Ultimate Life Table
- i = 0.05
- The gross annual premium, calculated using the equivalence principle, is of the form $G = gH + f$, where $g$ is the premium rate per 1 of insurance and $f$ is the per policy fee

Calculate $f$.

In [2]:
print("SOA Question 6.52:  (D) 50.80")
sult = SULT()
a = sult.temporary_annuity(45, t=10)
other_cost = 10 * sult.deferred_annuity(45, u=10)
P = sult.gross_premium(a=a, 
                       A=0, 
                       benefit=0, 
                       initial_premium=1.05, 
                       renewal_premium=0.05,
                       initial_policy=100 + other_cost, 
                       renewal_policy=20)
print(a, P)


SOA Question 6.52:  (D) 50.80
8.0750937741422 50.80135534704229


__SOA Question 6.47__

For a 10-year deferred whole life annuity-due with payments of 100,000 per year on (70), you are given:
- Annual gross premiums of $G$ are payable for 10 years
- First year expenses are 75% of premium
- Renewal expenses for years 2 and later are 5% of premium during the premium paying period
- Mortality follows the Standard Ultimate Life Table
- i = 0.05

Calculate $G$ using the equivalence principle.

In [3]:
print("SOA Question 6.47:  (D) 66400")
sult = SULT()
a = sult.temporary_annuity(70, t=10)
A = sult.deferred_annuity(70, u=10)
P = sult.gross_premium(a=a, A=A, benefit=100000, initial_premium=0.75,
                        renewal_premium=0.05)
print(P)


SOA Question 6.47:  (D) 66400
66384.13293704337


__SOA Question 6.43__

For a fully discrete, 5-payment 10-year term insurance of 200,000 on (30), you are given:
- Mortality follows the Standard Ultimate Life Table
- The following expenses are incurred at the beginning of each respective year:

| | Percent of Premium | Per Policy | Percent of Premium | Per Policy |
|---|---|---|---|---|
| | Year 1 | Year 1 | Years 2 - 10 | Years 2 - 10 |
| Taxes | 5% | 0 | 5% | 0 |
| Commissions | 30% | 0 | 10% | 0 |
| Maintenance | 0% | 8 | 0% | 4 |

- i = 0.05
- $\ddot{a}_{30:\overline{5|}} = 4.5431$

Calculate the annual gross premium using the equivalence principle.

In [4]:
print("SOA Question 6.43:  (C) 170")
sult = SULT()
a = sult.temporary_annuity(30, t=5)
A = sult.term_insurance(30, t=10)
other_expenses = 4 * sult.deferred_annuity(30, u=5, t=5)
P = sult.gross_premium(a=a, A=A, benefit=200000, initial_premium=0.35,
                        initial_policy=8 + other_expenses, renewal_policy=4,
                        renewal_premium=0.15)
print(P)


SOA Question 6.43:  (C) 170
171.22371939459944


__SOA Question 6.39__

XYZ Insurance writes 10,000 fully discrete whole life insurance policies of 1000 on lives age 40 and an additional 10,000 fully discrete whole life policies of 1000 on lives age 80.

XYZ used the following assumptions to determine the net premiums for these policies:

- Mortality follows the Standard Ultimate Life Table
- i = 0.05

During the first ten years, mortality did follow the Standard Ultimate Life Table.

Calculate the average net premium per policy in force received at the beginning of the eleventh year.

In [5]:
print("SOA Question 6.39:  (A) 29")
sult = SULT()
P40 = sult.premium_equivalence(sult.whole_life_insurance(40), b=1000)
P80 = sult.premium_equivalence(sult.whole_life_insurance(80), b=1000)
p40 = sult.p_x(40, t=10)
p80 = sult.p_x(80, t=10)
P = (P40 * p40 + P80 * p80) / (p80 + p40)
print(P)


SOA Question 6.39:  (A) 29
29.033866427845496


__SOA Question 6.37__

For a fully discrete whole life insurance policy of 50,000 on (35), with premiums payable for a maximum of 10 years, you are given:

- Expenses of 100 are payable at the end of each year including the year of death
- Mortality follows the Standard Ultimate Life Table
- i = 0.05

Calculate the annual gross premium using the equivalence principle.

In [6]:
print("SOA Question 6.37:  (D) 820")
sult = SULT()
benefits = sult.whole_life_insurance(35, b=50000 + 100)
expenses = sult.immediate_annuity(35, b=100)
a = sult.temporary_annuity(35, t=10)
print(benefits, expenses, a)
print((benefits + expenses) / a)


SOA Question 6.37:  (D) 820
4836.382819496279 1797.2773668474615 8.092602358383987
819.7190338249138


__SOA Question 6.35__

For a fully discrete whole life insurance policy of 100,000 on (35), you are given:
- First year commissions are 19% of the annual gross premium
- Renewal year commissions are 4% of the annual gross premium
- Mortality follows the Standard Ultimate Life Table
- i = 0.05

Calculate the annual gross premium for this policy using the equivalence principle.

In [7]:
print("SOA Question 6.35:  (D) 530")
sult = SULT()
A = sult.whole_life_insurance(35, b=100000)
a = sult.whole_life_annuity(35)
print(sult.gross_premium(a=a, A=A, initial_premium=.19, renewal_premium=.04))


SOA Question 6.35:  (D) 530
534.4072234303344


__SOA Question 5.8__

For an annual whole life annuity-due of 1 with a 5-year certain period on (55), you are given:
- Mortality follows the Standard Ultimate Life Table
- i = 0.05

Calculate the probability that the sum of the undiscounted payments actually made under this annuity will exceed the expected present value, at issue, of the annuity.

In [8]:
print("SOA Question 5.8: (C) 0.92118")
sult = SULT()
a = sult.certain_life_annuity(55, u=5)
print(sult.p_x(55, t=math.floor(a)))


SOA Question 5.8: (C) 0.92118
0.9211799771029529


__SOA Question 5.3__

You are given:

- Mortality follows the Standard Ultimate Life Table
- Deaths are uniformly distributed over each year of age
- i = 0.05

Calculate
$\frac{d}{dt}(\overline{I}\overline{a})_{40:\overline{t|}}$ at $t = 10.5$.

In [9]:
print("SOA Question 5.3:  (C) 6.239")
sult = SULT()
t = 10.5
print(t * sult.E_r(40, t=t))


SOA Question 5.3:  (C) 6.239
6.23871918627528


__SOA Question 4.17__

For a special whole life policy on (48), you are given:

- The policy pays 5000 if the insured’s death is before the median curtate future
lifetime at issue and 10,000 if death is after the median curtate future lifetime at issue
- Mortality follows the Standard Ultimate Life Table
- Death benefits are paid at the end of the year of death
- i = 0.05

Calculate the actuarial present value of benefits for this policy.

In [10]:
print("SOA Question 4.17:  (A) 1126.7")
sult = SULT()
median = sult.Z_t(48, prob=0.5, discrete=False)
benefit = lambda x,t: 5000 if t < median else 10000
print(sult.A_x(48, benefit=benefit))


SOA Question 4.17:  (A) 1126.7
1126.774772894844


__SOA Question 4.14__

A fund is established for the benefit of 400 workers all age 60 with independent future lifetimes. When they reach age 85, the fund will be dissolved and distributed to the survivors.

The fund will earn interest at a rate of 5% per year.

The initial fund balance, $F$, is determined so that the probability that the fund will pay at least 5000 to each survivor is 86%, using the normal approximation.

Mortality follows the Standard Ultimate Life Table.

Calculate $F$.

In [11]:
print("SOA Question 4.14:  (E) 390000    ")
sult = SULT()
p = sult.p_x(60, t=85-60)
mean = sult.bernoulli(p)
var = sult.bernoulli(p, variance=True)
F = sult.portfolio_percentile(mean=mean, variance=var, prob=.86, N=400)
print(F * 5000 * sult.interest.v_t(85-60))


SOA Question 4.14:  (E) 390000    
389322.86778416135


__SOA Question 4.5__

For a 30-year term life insurance of 100,000 on (45), you are given:
- The death benefit is payable at the moment of death
- Mortality follows the Standard Ultimate Life Table
- $\delta = 0.05$
- Deaths are uniformly distributed over each year of age

Calculate the 95th percentile of the present value of benefits random variable for this insurance

In [12]:
print("SOA Question 4.5:  (C) 35200")
sult = SULT(udd=True).set_interest(delta=0.05)
Z = 100000 * sult.Z_from_prob(45, prob=0.95, discrete=False)
print(Z)

SOA Question 4.5:  (C) 35200
35187.952037196534


__SOA Question 3.9__

A father-son club has 4000 members, 2000 of which are age 20 and the other 2000 are age 45. In 25 years, the members of the club intend to hold a reunion.
You are given:
- All lives have independent future lifetimes.
- Mortality follows the Standard Ultimate Life Table.

Using the normal approximation, without the continuity correction, calculate the 99th percentile of the number of surviving members at the time of the reunion.

In [13]:
print("SOA Question 3.9:  (E) 3850")
sult = SULT()
p1 = sult.p_x(20, t=25)
p2 = sult.p_x(45, t=25)
mean = sult.bernoulli(p1) * 2000 + sult.bernoulli(p2) * 2000
var = (sult.bernoulli(p1, variance=True) * 2000 
        + sult.bernoulli(p2, variance=True) * 2000)
print(sult.portfolio_percentile(mean=mean, variance=var, prob=.99))


SOA Question 3.9:  (E) 3850
3850.144345130047


__SOA Question 3.8__

A club is established with 2000 members, 1000 of exact age 35 and 1000 of exact age 45. You are given:
- Mortality follows the Standard Ultimate Life Table
- Future lifetimes are independent
- N is the random variable for the number of members still alive 40 years after the club is established

Using the normal approximation, without the continuity correction, calculate the smallest $n$ such that $Pr( N \ge n ) \le 0.05$.

In [14]:
print("SOA Question 3.8:  (B) 1505")
sult = SULT()
p1 = sult.p_x(35, t=40)
p2 = sult.p_x(45, t=40)
mean = sult.bernoulli(p1) * 1000 + sult.bernoulli(p2) * 1000
var = (sult.bernoulli(p1, variance=True) * 1000 
        + sult.bernoulli(p2, variance=True) * 1000)
print(sult.portfolio_percentile(mean=mean, variance=var, prob=.95))


SOA Question 3.8:  (B) 1505
1504.8328375406456


__SOA Question 3.4__

The SULT Club has 4000 members all age 25 with independent future lifetimes. The
mortality for each member follows the Standard Ultimate Life Table.

Calculate the largest integer N, using the normal approximation, such that the probability that there are at least N survivors at age 95 is at least 90%.

In [15]:
print("SOA Question 3.4:  (B) 815")
sult = SULT()
mean = sult.p_x(25, t=95-25)
var = sult.bernoulli(mean, variance=True)
print(sult.portfolio_percentile(N=4000, mean=mean, variance=var, prob=.1))

SOA Question 3.4:  (B) 815
815.0943255167722


__Generate SULT Table__:

In [19]:

import pandas as pd
print("Standard Ultimate Life Table at i=0.05")
pd.set_option('display.max_rows', None)
sult.frame()


Standard Ultimate Life Table at i=0.05


,l_x,q_x,a_x,A_x,2A_x,a_x:10,A_x:10,a_x:20,A_x:20,5_E_x,10_E_x,20_E_x
20,100000.0,0.000250,19.9664,0.04922,0.00580,8.0991,0.61433,13.0559,0.37829,0.78252,0.61224,0.37440
21,99975.0,0.000253,19.9197,0.05144,0.00614,8.0990,0.61433,13.0551,0.37833,0.78250,0.61220,0.37429
22,99949.7,0.000257,19.8707,0.05378,0.00652,8.0988,0.61434,13.0541,0.37837,0.78248,0.61215,0.37417
23,99924.0,0.000262,19.8193,0.05622,0.00694,8.0986,0.61435,13.0531,0.37842,0.78245,0.61210,0.37404
24,99897.8,0.000267,19.7655,0.05879,0.00739,8.0983,0.61437,13.0519,0.37848,0.78243,0.61205,0.37390
25,99871.1,0.000273,19.7090,0.06147,0.00788,8.0981,0.61438,13.0506,0.37854,0.78240,0.61198,0.37373
26,99843.8,0.000280,19.6499,0.06429,0.00841,8.0978,0.61439,13.0491,0.37862,0.78236,0.61191,0.37354
27,99815.9,0.000287,19.5878,0.06725,0.00900,8.0974,0.61441,13.0474,0.37869,0.78233,0.61183,0.37334
28,99787.2,0.000296,19.5228,0.07034,0.00964,8.0970,0.61443,13.0455,0.37878,0.78229,0.61174,0.37310
29,99757.7,0.000305,19.4547,0.07359,0.01033,8.0966,0.61445,13.0434,0.37888,0.78224,0.61163,0.37284
